In [3]:
# Random Local Search
# -------------------

import math
import random

x = -2
y = 3

def forwardMultiplyGate(x,y):
    return x*y

tweak_amount = 0.01
best_out = -1000000000
best_x = x
best_y = y

for k in range(100):
    x_try = x + tweak_amount * (random.random() * 2 - 1) # random val between -1 and 1
    y_try = y + tweak_amount * (random.random() * 2 - 1) # random val between -1 and 1
    out = forwardMultiplyGate(x_try, y_try)
    if out > best_out:
        best_out = out
        best_x = x_try
        best_y = y_try
        print(best_out)

-6.019520718059613
-6.016661205026821
-6.015469933355874
-5.995412301597359
-5.952624179201573


In [4]:
# Numerical Gradient
# ------------------

def forwardMultiplyGate(x, y):
    return x * y

x = -2
y = 3
out = forwardMultiplyGate(x, y)
h = 0.0001

xph = x + h
out2 = forwardMultiplyGate(xph, y)
x_derivative = (out2 - out) / h # Change WRT x, divided by the change

yph = y + h
out3 = forwardMultiplyGate(x, yph)
y_derivative = (out3 - out) / h # Change WRT y, divided by the change

print("x_derivative = ", x_derivative)
print("y_derivative = ", y_derivative)

gradient = [x_derivative, y_derivative]

step_size = 0.01

x = x + (step_size * x_derivative)
y = y + (step_size * y_derivative)
out_new = forwardMultiplyGate(x, y)

print("The new output is ", out_new)

x_derivative =  3.00000000000189
y_derivative =  -2.0000000000042206
The new output is  -5.87059999999986


In [5]:
# Analytic Gradient
# -----------------

x = -2
y = 3
out = forwardMultiplyGate(x, y)
x_gradient = y
y_gradient = x

step_size = 0.01
x += step_size * x_gradient
y += step_size * y_gradient
out_new = forwardMultiplyGate(x, y)

print("New output = ", out_new)

New output =  -5.8706


In [6]:
# Circuit
# -------

def forwardMultiplyGate(a, b):
    return a * b

def forwardAddGate(a, b):
    return a + b

def forwardCircuit(x, y, z):
    q = forwardAddGate(x, y)
    f = forwardMultiplyGate(q, z)
    return f

x = -2
y = 5
z = -4
q = forwardAddGate(x, y)
f = forwardMultiplyGate(q, z)

print("f = ", f)

# Derivative of the MULTIPLY gate
derivative_f_wrt_z = q
derivative_f_wrt_q = z

# Derivative of the ADD gate
derivative_q_wrt_x = 1.0
derivative_q_wrt_y = 1.0

# Chain rule
derivative_f_wrt_x = derivative_q_wrt_x * derivative_f_wrt_q
derivative_f_wrt_y = derivative_q_wrt_y * derivative_f_wrt_q

# Final gradient
gradient_f_wrt_xyz = [derivative_f_wrt_x, derivative_f_wrt_y, derivative_f_wrt_z]

# Let the inputs respond to the manipulations:
step_size = 0.01
x = x + step_size * derivative_f_wrt_x
y = y + step_size * derivative_f_wrt_y
z = z + step_size * derivative_f_wrt_z

# New circuit outputs
q = forwardAddGate(x, y)
f = forwardMultiplyGate(q, z)

print("f = ", f)

# Numerical Gradient CHECK with initial conditions

x = -2
y = 5
z = -4

h = 0.0001
x_derivative = (forwardCircuit(x+h, y, z) - forwardCircuit(x, y, z)) / h
y_derivative = (forwardCircuit(x, y+h, z) - forwardCircuit(x, y, z)) / h
z_derivative = (forwardCircuit(x, y, z+h) - forwardCircuit(x, y, z)) / h

print("Gradient: wrt_x = ", x_derivative, ", wrt_y = ", y_derivative, ", wrt_z = ", z_derivative)
print("Analytic Gradient : ", gradient_f_wrt_xyz)
print("Close enough!")

f =  -12
f =  -11.5924
Gradient: wrt_x =  -3.9999999999906777 , wrt_y =  -3.9999999999906777 , wrt_z =  3.000000000010772
Analytic Gradient :  [-4.0, -4.0, 3]
Close enough!


In [7]:
import math

class Unit:
    def __init__(self, value, grad):
        self.value = value
        self.grad = grad
        
class MultiplyGate:
    def __init__(self):
        pass
    
    def forward(self, u0_in, u1_in):
        """
        FORWARD PASS
        
        Creates an output unit (u_out) based on the input units
        """
        self.u0_in = u0_in
        self.u1_in = u1_in
        self.u_out = Unit(u0_in.value * u1_in.value, 0.0) # Initial gradient = 0.0
        return self.u_out
    
    def backward(self):
        """
        BACK PROPOGATION
        
        Adjusts the input gradients based on the outputs gradient
        """
        self.u0_in.grad += self.u1_in.value * self.u_out.grad
        self.u1_in.grad += self.u0_in.value * self.u_out.grad
        
class AddGate:
    def __init__(self):
        pass
        
    def forward(self, u0_in, u1_in):
        """
        FORWARD PASS
        
        Creates an output unit (u_out) based on the input units
        """
        self.u0_in = u0_in
        self.u1_in = u1_in
        self.u_out = Unit(u0_in.value + u1_in.value, 0.0) # Initial gradient = 0.0
        return self.u_out
    
    def backward(self):
        """
        BACK PROPOGATION
        
        Adjusts the input gradients based on the outputs gradient
        """
        self.u0_in.grad += 1 * self.u_out.grad
        self.u1_in.grad += 1 * self.u_out.grad
        
class SigmoidGate:
    def __init__(self):
        pass
    
    def sig(self, x):
        return 1 / (1 + math.exp(-x))
    
    def forward(self, u0_in):
        """
        FORWARD PASS
        
        Creates an output unit (u_out) based on the input units
        """
        self.u0_in = u0_in
        self.u_out = Unit(self.sig(self.u0_in.value), 0.0)
        return self.u_out
    
    def backward(self):
        """
        BACK PROPOGATION
        
        Adjusts the input gradients based on the outputs gradient
        """
        s = self.sig(self.u0_in.value)
        self.u0_in.grad += (s * (1 - s)) * self.u_out.grad

In [8]:
# Input Units

a = Unit(1.0, 0.0)
b = Unit(2.0, 0.0)
c = Unit(-3.0, 0.0)
x = Unit(-1.0, 0.0)
y = Unit(3.0, 0.0)

# Gates
mulg0 = MultiplyGate()
mulg1 = MultiplyGate()
addg0 = AddGate()
addg1 = AddGate()
sg0 = SigmoidGate()

def forwardNeuron():
    ax = mulg0.forward(a, x)
    by = mulg1.forward(b, y)
    axpby = addg0.forward(ax, by)
    axpbypc = addg1.forward(axpby, c) # a*x + b*y + c
    s = sg0.forward(axpbypc)
    
    return s
    
s = forwardNeuron()

print("Circuit output s = ", s.value)

# Computing the GRADIENT (backward func in reverse order to above):

s.grad = 1.0 # Initialise gradient to 1
sg0.backward()
addg1.backward()
addg0.backward()
mulg1.backward()
mulg0.backward()

# Make the inputs respond to the computed gradients

setp_size = 0.01
a.value += step_size * a.grad
b.value += step_size * b.grad
c.value += step_size * c.grad
x.value += step_size * x.grad
y.value += step_size * y.grad

backprop_gradients = [a.grad, b.grad, c.grad, x.grad, y.grad]

print(backprop_gradients)

s_new = forwardNeuron()

print('Circuit output after one backprop = ', s_new.value)

Circuit output s =  0.8807970779778823
[-0.10499358540350662, 0.31498075621051985, 0.10499358540350662, 0.10499358540350662, 0.20998717080701323]
Circuit output after one backprop =  0.8825501816218984


In [9]:
def forwardCircuitFast(a, b, c, x, y):
    return 1 / (1 + math.exp(- (a*x + b*y + c)))

a = 1
b = 2
c = -3
x = -1
y = 3
h = 0.0001

a_grad = (forwardCircuitFast(a+h, b, c, x, y) - forwardCircuitFast(a, b, c, x, y)) / h
b_grad = (forwardCircuitFast(a, b+h, c, x, y) - forwardCircuitFast(a, b, c, x, y)) / h
c_grad = (forwardCircuitFast(a, b, c+h, x, y) - forwardCircuitFast(a, b, c, x, y)) / h
x_grad = (forwardCircuitFast(a, b, c, x+h, y) - forwardCircuitFast(a, b, c, x, y)) / h
y_grad = (forwardCircuitFast(a, b, c, x, y+h) - forwardCircuitFast(a, b, c, x, y)) / h

FCF_gradient = [a_grad, b_grad, c_grad, x_grad, y_grad]

print(FCF_gradient) # VERY similar to the backprop_gradients above!

[-0.10499758359205913, 0.3149447748351797, 0.10498958734506125, 0.10498958734506125, 0.2099711788272618]


In [10]:
# Support Vector Machine

In [95]:
class Circuit:
    def __init__(self):
        self.mulg0 = MultiplyGate()
        self.mulg1 = MultiplyGate()
        self.addg0 = AddGate()
        self.addg1 = AddGate()
    
    def forward(self, x, y, a, b, c):
        ax = self.mulg0.forward(a, x)
        by = self.mulg1.forward(b, y)
        ax_plus_by = self.addg0.forward(ax, by)
        self.ax_plus_by_plus_c = self.addg1.forward(ax_plus_by, c)
        return self.ax_plus_by_plus_c
    
    def backward(self, gradient_top):
        self.ax_plus_by_plus_c.grad = gradient_top
        self.addg1.backward()
        self.addg0.backward()
        self.mulg1.backward()
        self.mulg0.backward()

In [127]:
class SVM:

    def __init__(self):
        self.a = Unit(1.0, 0.0)
        self.b = Unit(-2.0, 0.0)
        self.c = Unit(-1.0, 0.0)
        self.circuit = Circuit()
    
    def forward(self, x, y):
        self.unit_out = self.circuit.forward(x, y, self.a, self.b, self.c)
        return self.unit_out
    
    def backward(self, label):
        self.a.grad = 0.0
        self.b.grad = 0.0
        self.c.grad = 0.0
        
        pull = 0.0
        
        if(label == 1 and self.unit_out.value < 1):
            pull = 1
        if(label == -1 and self.unit_out.value > -1):
            pull = -1
        
        self.circuit.backward(pull)
        
        self.a.grad += -self.a.value
        self.b.grad += -self.b.value
        
    def learnFrom(self, x, y, label):
        self.forward(x, y)
        self.backward(label)
        self.parameterUpdate()
        
    def parameterUpdate(self):
        step_size = 0.01
        self.a.value += step_size * self.a.grad
        self.b.value += step_size * self.b.grad
        self.c.value += step_size * self.c.grad

In [128]:
data   = [[1.2, 0.7], [-0.3, -0.5], [3.0, 0.1], [-0.1, -1.0], [-1.0, 1.1], [2.1, -3]]
labels = [1, -1, 1, -1, -1, 1]

svm = SVM()

def evalTrainingAccuracy():
    num_correct = 0
    for i in range(len(data)):
        x = Unit(data[i][0], 0.0)
        y = Unit(data[i][1], 0.0)
        true_label = labels[i]
        
        predicted_label = 1 if (svm.forward(x,y).value > 0) else -1
        
        if(predicted_label == true_label):
            num_correct += 1
            
    return num_correct / len(data)

for it in range(10000):
    i = math.floor(random.random() * len(data))
    x = Unit(data[i][0], 0.0)
    y = Unit(data[i][1], 0.0)
    label = labels[i]
    svm.learnFrom(x,y,label)
    
    #if(it%1000==0):
    #    print("Training accuracy at iter {0}: {1}".format(it, evalTrainingAccuracy()))

```
Training accuracy at iter 0: 0.6666666666666666
Training accuracy at iter 25: 0.6666666666666666
Training accuracy at iter 50: 0.8333333333333334
Training accuracy at iter 75: 0.8333333333333334
Training accuracy at iter 100: 0.8333333333333334
Training accuracy at iter 125: 0.8333333333333334
Training accuracy at iter 150: 0.8333333333333334
Training accuracy at iter 175: 0.8333333333333334
Training accuracy at iter 200: 0.8333333333333334
Training accuracy at iter 225: 0.8333333333333334
Training accuracy at iter 250: 0.8333333333333334
Training accuracy at iter 275: 0.8333333333333334
Training accuracy at iter 300: 0.8333333333333334
Training accuracy at iter 325: 0.8333333333333334
Training accuracy at iter 350: 0.8333333333333334
Training accuracy at iter 375: 1.0
```

```
Training accuracy at iter 0: 0.6666666666666666
Training accuracy at iter 1000: 0.8333333333333334
Training accuracy at iter 2000: 1.0
Training accuracy at iter 3000: 1.0
Training accuracy at iter 4000: 0.8333333333333334
Training accuracy at iter 5000: 1.0
Training accuracy at iter 6000: 0.8333333333333334
Training accuracy at iter 7000: 0.8333333333333334
Training accuracy at iter 8000: 0.8333333333333334
Training accuracy at iter 9000: 0.8333333333333334
```

In [153]:
# Condensed Form:

a = 1
b = -2
c = -1

def evalTrainingAccuracy(a, b, c):
    num_correct = 0
    for i in range(len(data)):
        x = data[i][0]
        y = data[i][1]
        true_label = labels[i]
        
        score = a*x + b*y + c

        predicted_label = 1 if (score > 0) else -1
        
        
        if(predicted_label == true_label):
            num_correct += 1
    
        
    return num_correct / len(data)

for it in range(10000):
    i = math.floor(random.random()*len(data))
    x = data[i][0]
    y = data[i][1]
    label = labels[i]
    
    score = a*x + b*y + c
    pull = 0.0
    if label == 1 and score < 1:
        pull = 1 
    if label == -1 and score > -1:
        pull = -1 
    
    step_size = 0.01
    a += step_size * (x*pull-a)
    b += step_size * (y*pull-b)
    c += step_size * (pull)
    
    if(it%1000==0):
        # print("i: {0}, data: {1}, label: {2}".format(i, data[i], label))
        # print("score: {0}".format(score))
        # print("x: {3}, y: {4}, a: {0}, b: {1}, c: {2}".format(a, b, c, x, y))
        print("Training accuracy at iter {0}: {1}".format(it, evalTrainingAccuracy(a, b, c)))

Training accuracy at iter 0: 0.6666666666666666
Training accuracy at iter 1000: 1.0
Training accuracy at iter 2000: 1.0
Training accuracy at iter 3000: 1.0
Training accuracy at iter 4000: 0.8333333333333334
Training accuracy at iter 5000: 1.0
Training accuracy at iter 6000: 1.0
Training accuracy at iter 7000: 1.0
Training accuracy at iter 8000: 0.8333333333333334
Training accuracy at iter 9000: 1.0


In [147]:
data   = [[1.2, 0.7], [-0.3, -0.5], [3.0, 0.1], [-0.1, -1.0], [-1.0, 1.1], [2.1, -3]]
labels = [1, -1, 1, -1, -1, 1]

a1 = random.random() - 0.5;
b1 = random.random() - 0.5;
c1 = random.random() - 0.5;
d1 = random.random() - 0.5;
a2 = random.random() - 0.5;
b2 = random.random() - 0.5;
c2 = random.random() - 0.5;
d2 = random.random() - 0.5;
a3 = random.random() - 0.5;
b3 = random.random() - 0.5;
c3 = random.random() - 0.5;
d3 = random.random() - 0.5;
a4 = random.random() - 0.5;
b4 = random.random() - 0.5;
c4 = random.random() - 0.5;
d4 = random.random() - 0.5;

def evalTrainingAccuracy(a1, b1, c1, a2, b2, c2, a3, b3, c3, a4, b4, c4, d4):
    num_correct = 0
    for i in range(len(data)):
        x = data[i][0]
        y = data[i][1]
        true_label = labels[i]
        
        n1 = max(0, a1*x + b1*y + c1)
        n2 = max(0, a2*x + b2*y + c2)
        n3 = max(0, a3*x + b3*y + c3)
        score = a4*n1 + b4*n2 + c4*n3 + d4

        predicted_label = 1 if (score > 0) else -1
        
        if(predicted_label == true_label):
            num_correct += 1
    
        
    return num_correct / len(data)

for it in range(400):
    i = math.floor(random.random()*len(data))
    x = data[i][0]
    y = data[i][1]
    label = labels[i]
    
    # Forward Pass
    n1 = max(0, a1*x + b1*y + c1)
    n2 = max(0, a2*x + b2*y + c2)
    n3 = max(0, a3*x + b3*y + c3)
    score = a4*n1 + b4*n2 +c4*n3 + d4
    
    pull = 0
    if (label == 1 and score < 1):
        pull = 1 
    if (label == -1 and score > -1):
        pull = -1
        
    dscore = pull
    da4 = n1 * dscore
    dn1 = a4 * dscore
    db4 = n2 * dscore
    dn2 = b4 * dscore
    dc4 = n3 * dscore
    dn3 = c4 * dscore
    dd4 = 1 * dscore
    
    # Backprop the ReLU nonlinearities in place
    
    dn3 = 0 if n3 == 0 else dn3
    dn2 = 0 if n2 == 0 else dn2
    dn1 = 0 if n1 == 0 else dn1
    
    da1 = x * dn1
    db1 = y * dn1
    dc1 = dn1
    
    da2 = x * dn2
    db2 = y * dn2
    dc2 = dn2
    
    da3 = x * dn3
    db3 = y * dn3
    dc3 = dn3
    
    da1 += -a1
    da2 += -a2
    da3 += -a3
    db1 += -b1
    db2 += -b2
    db3 += -b3
    da4 += -a4
    db4 += -b4
    dc4 += -c4
    
    step_size = 0.01;
    a1 += step_size * da1
    b1 += step_size * db1 
    c1 += step_size * dc1
    a2 += step_size * da2 
    b2 += step_size * db2
    c2 += step_size * dc2
    a3 += step_size * da3 
    b3 += step_size * db3 
    c3 += step_size * dc3
    a4 += step_size * da4 
    b4 += step_size * db4 
    c4 += step_size * dc4 
    d4 += step_size * dd4
    
    if(it%25==0):
        # print("i: {0}, data: {1}, label: {2}".format(i, data[i], label))
        # print("score: {0}".format(score))
        # print("x: {3}, y: {4}, a: {0}, b: {1}, c: {2}".format(a, b, c, x, y))
        # print(dn3, dn2, dn1)
        print("Training accuracy at iter {0}: {1}".format(it, evalTrainingAccuracy(a1, b1, c1, a2, b2, c2, a3, b3, c3, a4, b4, c4, d4)))

Training accuracy at iter 0: 0.3333333333333333
Training accuracy at iter 25: 0.3333333333333333
Training accuracy at iter 50: 0.5
Training accuracy at iter 75: 0.5
Training accuracy at iter 100: 0.6666666666666666
Training accuracy at iter 125: 0.8333333333333334
Training accuracy at iter 150: 0.6666666666666666
Training accuracy at iter 175: 0.8333333333333334
Training accuracy at iter 200: 1.0
Training accuracy at iter 225: 0.8333333333333334
Training accuracy at iter 250: 1.0
Training accuracy at iter 275: 0.8333333333333334
Training accuracy at iter 300: 0.8333333333333334
Training accuracy at iter 325: 0.8333333333333334
Training accuracy at iter 350: 1.0
Training accuracy at iter 375: 1.0


```
Training accuracy at iter 0: 0.3333333333333333
Training accuracy at iter 25: 0.3333333333333333
Training accuracy at iter 50: 0.5
Training accuracy at iter 75: 0.5
Training accuracy at iter 100: 0.6666666666666666
Training accuracy at iter 125: 0.8333333333333334
Training accuracy at iter 150: 0.6666666666666666
Training accuracy at iter 175: 0.8333333333333334
Training accuracy at iter 200: 1.0
Training accuracy at iter 225: 0.8333333333333334
Training accuracy at iter 250: 1.0
Training accuracy at iter 275: 0.8333333333333334
Training accuracy at iter 300: 0.8333333333333334
Training accuracy at iter 325: 0.8333333333333334
Training accuracy at iter 350: 1.0
Training accuracy at iter 375: 1.0
```